In [ ]:
import astropy.units as u
import click
import fastimgproto.casa.io as casa_io
import fastimgproto.casa.simulation as casa_sim
import fastimgproto.imager as imager
import fastimgproto.visibility as visibility
from astropy.coordinates import Angle, SkyCoord
from fastimgproto.gridder.conv_funcs import GaussianSinc
from fastimgproto.skymodel.helpers import SkyRegion, SkySource
from fastimgproto.sourcefind.image import SourceFindImage
import numpy as np
import logging
import os

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# Plot image pixels in cartesian ordering (i.e. y-positive == upwards):
plt.rcParams['image.origin'] = 'lower'
# Make plots bigger
plt.rcParams['figure.figsize'] = 10, 10

In [ ]:
casavis_path = 'vis.ms'
uvw_path = 'uvw.npz'
load_uvw = False
image_size=1024 * u.pixel
cell_size=3 * u.arcsecond


In [ ]:
logging.basicConfig(level=logging.DEBUG)

pointing_centre = SkyCoord(180 * u.deg, 8 * u.deg)
field_of_view = SkyRegion(pointing_centre,
                          radius=Angle(1 * u.deg))

# source_list = get_lsm(field_of_view)
# source_list = get_spiral_source_test_pattern(field_of_view)
northeast_of_centre = SkyCoord(
    ra=pointing_centre.ra + 0.01 * u.deg,
    dec=pointing_centre.dec + 0.01 * u.deg, )
steady_source_list = [
    SkySource(position=pointing_centre, flux=1 * u.Jy),
    SkySource(position=northeast_of_centre, flux=0.4 * u.Jy),
]

southwest_of_centre = SkyCoord(
    ra=field_of_view.centre.ra - 0.05 * u.deg,
    dec=field_of_view.centre.dec - 0.05 * u.deg)
transient_source_list = [
    SkySource(position=southwest_of_centre, flux=0.5 * u.Jy),
]

# Std. dev of Gaussian noise added to visibilities for each baseline:
# (Jointly normal, i.e. independently added to real / imaginary components.)
vis_noise_level = 0.001 * u.Jy

# Simulate visibilities using casapy to generate a set of UVW baselines.
# (This is next on the list for a 'from scratch' implementation,
# at which point the CASA / casacore dependency becomes purely optional, for
# cross-validation purposes.)
if not load_uvw:
    for path in uvw_path, casavis_path:
        output_dir = os.path.dirname(os.path.abspath(path))
        if not os.path.isdir(output_dir):
            os.makedirs(output_dir)
    casa_output = casa_sim.simulate_vis_with_casa(pointing_centre,
                                                  steady_source_list,
                                                  # source_list_w_transient,
                                                  noise_std_dev=vis_noise_level,
                                                  vis_path=casavis_path)
    uvw_lambda = casa_io.get_uvw_in_lambda(casavis_path)
    with open(uvw_path, 'wb') as f:
        np.savez(f, uvw_lambda=uvw_lambda)
else:
    with open(uvw_path, 'rb') as f:
        npz_content = np.load(f)
        uvw_lambda = npz_content['uvw_lambda']



In [ ]:
source_list_w_transient = steady_source_list + transient_source_list

# Now use UVW to generate visibilities from scratch...
# Represent incoming data; includes transient sources, noise:
data_vis = visibility.calculated_summed_vis(
    pointing_centre, source_list_w_transient, uvw_lambda)
data_vis = visibility.add_gaussian_noise(vis_noise_level, data_vis)

# Model vis; only steady sources from the catalog, noise free.
model_vis = visibility.calculated_summed_vis(
    pointing_centre, steady_source_list, uvw_lambda)

# Subtract model-generated visibilities from incoming data
residual_vis = data_vis - model_vis

# Will move this to a config option later
kernel_support = 3
kernel_func = GaussianSinc(trunc=kernel_support)


In [ ]:
data_image, beam = imager.image_visibilities(data_vis, uvw_lambda,
                                        image_size=image_size,
                                        cell_size=cell_size,
                                        kernel_func=kernel_func,
                                        kernel_support=kernel_support,
                                        kernel_oversampling=None)


In [ ]:
plt.imshow(np.real(data_image))

In [ ]:
resids_image, beam = imager.image_visibilities(residual_vis, uvw_lambda,
                                        image_size=image_size,
                                        cell_size=cell_size,
                                        kernel_func=kernel_func,
                                        kernel_support=kernel_support,
                                        kernel_oversampling=None)



In [ ]:
# detection_n_sigma=30
detection_n_sigma=50
analysis_n_sigma=25
sfimage = SourceFindImage(data=np.real(resids_image),
                          detection_n_sigma=detection_n_sigma,
                          analysis_n_sigma=analysis_n_sigma,
                          )

In [ ]:
sfimage.islands

In [ ]:
plt.imshow(sfimage.data)
for island in sfimage.islands:
#     plt.scatter(island.xbar,island.ybar, marker='*', s=100, c='y',)
    pass